# Basis representation delle windows e fANOVA con Hochberg

In [1]:
import numpy as np
import os
import pandas as pd
import mne
import skfda
import matplotlib.pyplot as plt
from skfda.exploratory.visualization import Boxplot
from skfda.inference.anova import oneway_anova

## Funzione per basis representation e per rimuovere gli outliers

In [2]:
def Basis_rep(cat,ch):
    df=pd.read_csv(home_path+'\\dataframes\\'+cat+'\\'+cat+'_'+ch+'.csv')
    data=df.iloc[:,2:]
    datagrid=skfda.FDataGrid(data_matrix=data.T)
    basis = skfda.representation.basis.BSpline(n_basis=18)
    smoother=skfda.preprocessing.smoothing.BasisSmoother(basis, smoothing_parameter=1e-5)
    cat_basis = smoother.fit_transform(datagrid)
    
    fdBoxplot = Boxplot(cat_basis.to_grid())
    non_outliers = [i for (i, b) in zip(np.arange(len(fdBoxplot.outliers)), fdBoxplot.outliers) if not b]
    cat_basis_no=cat_basis[non_outliers]
    outliers = [i for (i, b) in zip(np.arange(len(fdBoxplot.outliers)), fdBoxplot.outliers) if b]
    
    return cat_basis_no, outliers

## Hochberg

In [3]:
def reversed_enumerate(l):
    return zip(range(len(l)-1, -1, -1), reversed(l))

def Hochberg(p_val_passed, alpha = 0.05, debug = False):

    p_val = p_val_passed.copy() # copy of the list to not modify it

    m = len(p_val)
    
    order = p_val.argsort()
    p_val = p_val[order]

    if debug:
        print('m = ',m,'\norder: ',order,'\npval sorted: ',p_val,'\n\nFor loop: \n')

    for i,p in reversed_enumerate(p_val):
        treshold = alpha/(m - i)    # alpha \ (m - (i + 1) + 1)
        if debug:
            print('i ordered = ',order[i],'\tpval = ',p,'\t treshold = ',treshold)
        if(p <= treshold):
            break

    if i == 0:
        idx_reject = []
    else:
        idx_reject = order[:(i+1)]
    
    if debug:
        print('\nindexes rejected: ', idx_reject)

    return idx_reject

In [4]:
home_path = os.path.abspath(os.getcwd())

channels=['Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1',
          'Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6','CP2','C4','T8',
          'FT10','FC6','FC2','F4','F8','Fp2','AF7','AF3','AFz','F1','F5','FT7',
          'FC3','C1','C5','TP7','CP3','P1','P5','PO7','PO3','POz','PO4','PO8',
          'P6','P2','CPz','CP4','TP8','C6','C2','FC4','FT8','F6','AF8','AF4',
          'F2','FCz']

directory_csv=home_path + '\dataframes'

In [5]:
#Basis representation e fANOVA
p_val=pd.DataFrame(index=channels,columns=['p_val'])

for ch in channels:
    animal, an_out=Basis_rep('animal', ch)
    body, body_out=Basis_rep('body', ch)
    vehicle, v_out=Basis_rep('vehicle', ch)
    tool, t_out=Basis_rep('tool', ch)
    food, f_out=Basis_rep('food', ch)
    
    v_n, p = oneway_anova(animal, body, vehicle, tool, food)
    
    p_val.loc[ch]=p
    
p_val.to_csv('p_values_3.csv')

In [6]:
#Applico Hochberg

indxs_Hoch = Hochberg(p_val.to_numpy().flatten(), debug = True)
indxs_Hoch
    
ch_rej=list(channels[i] for i in np.sort(indxs_Hoch))
ch_rej

m =  63 
order:  [ 0 61 32 33 34 35 36 37 38 40 42 43 44 30 45 47 48 49 50 51 52 53 54 55
 58 59 60 46 29 31 27  1  2  3  4  5  6  7  8  9 10 11 12 28 14 13 16 26
 25 24 23 15 22 19 18 17 20 62 21 41 57 39 56] 
pval sorted:  [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0005 0.02 0.037 0.0765 0.327] 

For loop: 

i ordered =  56 	pval =  0.327 	 treshold =  0.05
i ordered =  39 	pval =  0.0765 	 treshold =  0.025
i ordered =  57 	pval =  0.037 	 treshold =  0.016666666666666666
i ordered =  41 	pval =  0.02 	 treshold =  0.0125
i ordered =  21 	pval =  0.0005 	 treshold =  0.01

indexes rejected:  [ 0 61 32 33 34 35 36 37 38 40 42 43 44 30 45 47 48 49 50 51 52 53 54 55
 58 59 60 46 29 31 27  1  2  3  4  5  6  7  8  9 10 11 12 28 14 13 16 26
 25 24 23 15 22 19 18 17 20 62 21]


['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'TP9',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'TP10',
 'CP6',
 'CP2',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'TP7',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'F6',
 'AF8',
 'AF4',
 'F2',
 'FCz']